In [1]:
import quimb as qu
import quimb.tensor as qtn
import matplotlib.pyplot as plt
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator
import json

/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
num_sites = 2
t1 = 1.
t2 = 0.5 # trying without the NN terms first, as additional check
U = 2.

In [3]:
num_qubits = num_sites*2

# read the file with the optimized MERA
name_file = f'not-optMERA_1x{num_sites}_U={U}_t2={t2}.json'
with open(name_file, 'r') as openfile:
    # Reading from json file
    mera_json = json.load(openfile)

# recovering the tensors
for item in mera_json['tensors']:
    print(item)
    print('-----------------------\n')



[[0.25463618391915877, 0.49736393359604136, 0.5240011711672311, -0.6428159176718731], [0.2650261322324709, 0.7452055032321766, -0.5731247914448934, 0.2143778920915695], [-0.9272017023614892, 0.3688746222680709, 0.02912187381640364, -0.05814148819876182], [0.07223964431627956, 0.24744572850817403, 0.6293668743364558, 0.7331093934196158]]
-----------------------

[[0.03821127311656324, -0.5047316638575556, 0.853873688624506, 0.12118403352411755], [-0.4200378958846649, 0.6182978141288009, 0.4532060575529726, -0.4856750441000875], [-0.6785720607077288, 0.11324780550687077, -0.005689257324995182, 0.7257289613387587], [0.6013717843850515, 0.5917167355170952, 0.2558742379044071, 0.47196573628958555]]
-----------------------

[[0.6338642129915022, 0.1024803620025143, 0.2436580412438019, -0.726873230921137], [0.04795109502509583, 0.6740900473752397, 0.6467153003882898, 0.35364193863175325], [0.20335720882866148, -0.7283145916628874, 0.5943046251209408, 0.2738717103831555], [0.7446896652915054, 

In [4]:
qc = QuantumCircuit(num_qubits) # initialized with all |0> states

nlayers = np.log2(2*num_sites)
num_gates = len(mera_json['tensors'])

for i in np.arange(num_gates-1, -1, -1):
    q = mera_json['qubits'][i]
    tn = mera_json['tensors'][i]

    # easy to add QEC gates here before/after the isometries!!
    op = Operator(tn)
    if i == num_gates-1:
        qc.unitary(op, [q[-1],q[0]], label=f'{mera_json['tags'][i]}_{i}')
    else:
        qc.unitary(op, q, label=f'{mera_json['tags'][i]}_{i}')

qc.draw('text')

┌────────┐          ┌────────┐
q_0: ────────────────────┤1       ├──────────┤0       ├
     ┌────────┐┌────────┐│        │┌────────┐│  UNI_0 │
q_1: ┤0       ├┤0       ├┤        ├┤0       ├┤1       ├
     │        ││        ││  ISO_3 ││  ISO_2 │├────────┤
q_2: ┤  ISO_5 ├┤  UNI_4 ├┤        ├┤1       ├┤0       ├
     │        ││        ││        │└────────┘│  UNI_1 │
q_3: ┤1       ├┤1       ├┤0       ├──────────┤1       ├
     └────────┘└────────┘└────────┘          └────────┘

In [5]:
# # the loop now needs to start from the end...
# # or I select first the pairs of qubits...

# list_qubits = []
# list_layer_tag = []
# jump = 1
# ind_q = 0
# while jump <= num_sites:
#     if ind_q < num_qubits-1 and ind_q + jump <= num_qubits-1:
#         list_qubits.append([ind_q, ind_q+jump])
#         list_layer_tag.append(jump-1)
#         ind_q += jump
#     else:
#         list_qubits.append([ind_q, ind_q-num_qubits+jump]) # CHECK THE ORDER HERE!!!!
#         # list_qubits.append([ind_q-num_qubits+jump, ind_q]) # CHECK THE ORDER HERE!!!!
#         list_layer_tag.append(jump-1)

#         jump += 1
#         ind_q = ind_q - num_qubits + jump + 1

# list_uni_iso = []
# for j in np.arange(len(mera_json)):
#     if j % 2 == 0:
#         list_uni_iso.append('UNI')
#     else:
#         list_uni_iso.append('ISO')

# print(list_qubits)
# print(list_layer_tag)
# print(list_uni_iso)

In [6]:
# # now I need to assign those tensors to gates in a circuit

# qc = QuantumCircuit(num_qubits) # initialized with all |0> states

# i = len(mera_json)-1

# while i >= 0:
#     if list_uni_iso[i] == 'ISO':
#         # and apply the gate right away
#         op = Operator(mera_json[i])
#         qc.unitary(op, list_qubits[i], label=f'ISO_{list_layer_tag[i]}')
#         i -= 1
#     else:
#         # apply first the next ISO if it is in the same layer!!, then this UNI --- NOOOO - I NEED TO APPLY FIRST ALL THE ISO OF A LAYER, THEN ALL THE UNITARIES!!!
#         if list_uni_iso[i-1] == 'ISO' and list_layer_tag[i-1] == list_layer_tag[i]:
#             op_iso = Operator(mera_json[i-1])
#             qc.unitary(op_iso, list_qubits[i-1], label=f'ISO_{list_layer_tag[i-1]}')

#             op = Operator(mera_json[i])
#             qc.unitary(op, list_qubits[i], label=f'UNI_{list_layer_tag[i]}')
#             i -= 2
#         else:
#             op = Operator(mera_json[i])
#             qc.unitary(op, list_qubits[i], label=f'UNI_{list_layer_tag[i]}')
#             i -= 1


# # nlayers = np.log2(2*num_sites)
# # i = len(mera_json)-1

# # for i_l in np.arange(nlayers-1, -1, -1):
# #     store_uni_tn = []
# #     store_uni_inds = []
# #     for i_tn in np.arange(len(mera_json)-1,-1,-1):
# #         if list_layer_tag[i_tn] == i_l:
# #             if list_uni_iso[i_tn] == 'UNI':
# #                 store_uni_tn.append(mera_json[i_tn])
# #                 store_uni_inds.append(list_qubits[i_tn])
# #             elif list_uni_iso[i_tn] == 'ISO':
# #                 op = Operator(mera_json[i_tn])
# #                 qc.unitary(op, list_qubits[i_tn], label=f'ISO_{list_layer_tag[i_tn]}')
    
# #     for q_uni, uni in zip(store_uni_inds, store_uni_tn):
# #         op = Operator(uni)
# #         qc.unitary(op, q_uni, label=f'UNI_{int(i_l)}')

# qc.draw('text')


In [7]:
# for i in np.arange(len(mera_json)-1, -1, -1):
#     print(i)
#     op_uni = Operator(mera_json[i])
#     if list_uni_iso[i] == 'UNI':
#         label = 'UNI'
#     else:
#         label = 'ISO'
#     qc.unitary(op, list_qubits[i], label=label)


# # add gates with unitaries(???) as the tensors we have -- the number of gates and position is fixed, as in the MERA (just reverse order )
# nlayers = np.log2(2*num_sites)
# id_gates = 0
# i_tensor = 0
# for i_l in np.arange(nlayers):
#     for i_q in np.arange(0,2*(int(num_sites-i_l)),2):
#         op = Operator()
#         qc.cx(i_q, i_q+1, 'UNI')
#         i_tensor += 1
#     for i_q in np.arange(0,2*(int(num_sites-i_l)),2):
#         if i_q+2 == 2*num_sites:
#             qc.cx(i_q+1, 0, 'ISO')
#         else:
#             qc.cx(i_q+1, i_q+2, 'ISO')
#         i_tensor += 1
        
#     qc.barrier()

# qc.draw(output="text")


Fidelity check

In [12]:
# define the Hamiltonian in qiskit -- it needs to contain also the terms with the chemical potential if I want to compare the energy...
from mera_hubbard import create_model_qiskit, solve_ground_state_qiskit, FH_Hamiltonian_NN_half_filling, solve_ground_state 
from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info import Statevector

lattice_type = [num_sites]
# automated - do not change
model_dim = len(lattice_type)
if model_dim == 1:
    num_sites_per_col = 1
    num_sites_per_row = lattice_type[0]
else:
    num_sites_per_col = lattice_type[0]
    num_sites_per_row = lattice_type[1]
num_sites = num_sites_per_col * num_sites_per_row

# additional check - hamiltonian in quimb
# FH HAMILTONIAN
ham, sparse_ham = FH_Hamiltonian_NN_half_filling(num_sites, t1, t2, U, pbc=0) # pbc=0: open BC, pbc=1: cyclic BC
ham.show()
en, _, v = solve_ground_state(num_sites, ham, sparse_ham)

Reference ED at half-filling:


/Users/bai/local/src/MERA-Hubbard/.venv/lib/python3.12/site-packages/quimb/experimental/operatorbuilder/operatorbuilder.py:1431: NumbaTypeSafetyWarning: unsafe cast from int64 to undefined. Precision may be lost.
  ci = bitmap[bi]


energy =  -1.2360679774997894
number of particles =  2.0


New Hamiltonian at half-filling:
chemical potential at half-filling =  -0.882
SparseOperatorBuilder(nsites=4, nterms=10, locality=2))
+ - . .  -1.0
- + . .  -1.0
. . + -  -1.0
. . - +  -1.0
n . n .  +2.0
sn. . .  -0.8820000000001231
. . sn.  -0.8820000000001231
. n . n  +2.0
. sn. .  -0.8820000000001231
. . . sn -0.8820000000001231
energy =  -1.2360679774997898
number of particles =  2.0


In [13]:
# create the model qiskit
mu = 0.
model, ham = create_model_qiskit(lattice_type, t1, U, mu, t2, draw=False)
en_qiskit, num_part_qiskit, eigenstate = solve_ground_state_qiskit(model, True)

# convert the circuit to its final wavefuction
psi = Statevector(qc)
state_fidelity(psi, eigenstate)

Energy gs =  -1.2360679774997891
Number of particles = 2.0
Magnetization = 0.0


0.006113175858698562

In [14]:
psi.draw('qsphere')

MissingOptionalLibraryError: "The 'seaborn' library is required to use 'plot_state_qsphere'. You can install it with 'pip install seaborn'."

In [ ]:
eigenstate.draw('qsphere')

NameError: name 'eigenstate' is not defined

In [25]:
# # calculate the energy
# psi.conjugate().inner(ham.compose(psi))

# # I need to convert the ham from FermionicOp to Operator.....